### Properties

To be clear, here we are examining **instance** properties. That is, we define the property in the class we are defining, but the property itself is going to be **instance** specific, i.e. different instances will support different values for the property. Just like instance attributes. The main difference is that we will use accessor method to get, set (and optionally) delete the associated instance value.

As I mentioned in the lecture, because properties use the same dotted notation (and the same `getattr`, `setattr` and `delattr` functions), we do not need to **start** with properties. Often a bare attribute works just fine, and if, later, we decide we need to manage getting/setting/deleting the attribute value, we can switch over to properties without breaking our class interface. This is unlike languages like Java - and hence why in those languages it is recommended to **always** use getter and setter functions. *Not so* in Python!

A **property** in Python is essentially a class instance - we'll come back to what that class looks like when we study descriptors. For now, we are going to use the `property` function in Python which is a convenience callable essentially.

Let's start with a simple example and a bare attribute:

In [1]:
class Person:
    def __init__(self, name):
        self.name = name

So this class has a single instance **attribute**, `name`.

In [2]:
p = Person('Ali')

And we can access and modify that attribute using either dotted notation or the `getattr` and `setattr` methods:

In [3]:
p.name

'Ali'

In [4]:
getattr(p, 'name')

'Ali'

In [5]:
p.name = 'Javad'

In [6]:
p.name

'Javad'

In [7]:
setattr(p, 'name', 'Milad')

In [8]:
p.name

'Milad'

Now suppose we wan't to disallow setting an empty string or `None` for the name. Also, we'll require the name to be a string.

To do that we are going to create an instance method that will handle the logic and setting of the value. We also create an instance method to retrieve the attribute value.

We'll use `_name` as the instance variable to store the name.

In [9]:
class Person:
    def __init__(self, name):
        self.set_name(name)
        # self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if isinstance(value, str) and len(value.strip()) > 0:
            # this is valid
            self._name = value.strip()
        else:
            raise ValueError('name must be a non-empty string')

In [10]:
p = Person('Ali')

In [11]:
p.set_name(100)

ValueError: name must be a non-empty string

In [15]:
try:
    p.set_name(100)
except:
    print('khata')
else:
    print('ok')
print('done')

khata
done


In [16]:
p.set_name('Amin')

In [17]:
p.get_name()

'Amin'

Of course, our users can still manipulate the atribute directly if they want by using the "private" attribute `_name`. You can't stop anyone from doing this in Python - they should know better than to do that, but we're all good programmers, and know what and what not to do!

The way we set up our initializer, the validation will work too:

In [18]:
try:
    p = Person('')
except ValueError as ex:
    print(ex)

name must be a non-empty string


So this works, but it's a bit of pain to use the method names. So let's turn this into a property instead. We start with the class we just had and tweak it a bit:

In [19]:
class Person:
    def __init__(self, name):
        self.name = name  # note how we are actually setting the value for name using the property!
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if isinstance(value, str) and len(value.strip()) > 0:
            # this is valid
            self._name = value.strip()
        else:
            raise ValueError('name must be a non-empty string')
            
    name = property(fget=get_name, fset=set_name)

In [20]:
p = Person('Ali')

In [21]:
p.name

'Ali'

In [22]:
p.name = 'Amin'

In [23]:
try:
    p.name = None
except ValueError as ex:
    print(ex)

name must be a non-empty string


In [25]:
p.__dict__

{'_name': 'Amin'}

So now we have the benefit of using accessor methods, without having to call the methods explicitly.

In fact, even `getattr` and `setattr` will work the same way:

In [26]:
setattr(p, 'name', 'John')  # or p.name = 'John'

In [27]:
getattr(p, 'name')  # or simply p.name

'John'

Now let's examine the instance dictionary:

In [28]:
p.__dict__

{'_name': 'John'}

You'll see we can find the underlying "private" attribute we are using to store the name. But the property itself (`name`) is not in the dictionary.

The property was defined in the class:

In [29]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'get_name': <function __main__.Person.get_name(self)>,
              'set_name': <function __main__.Person.set_name(self, value)>,
              'name': <property at 0x1805a470810>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

And you can see it's type is `property`.

So when we type `p.name` or `p.name = value`, Python recognizes that `'name` is a `property` and therefore uses the accessor methods. (How it does we'll see later when we study descriptors).

What's interesting is that even if we muck around with the instance dictionary, Python does not get confused - (and as usual in Python, just because you **can** do something does not mean you **should**!)

In [30]:
p = Person('Alex')

In [31]:
p.name

'Alex'

In [32]:
p.__dict__

{'_name': 'Alex'}

In [33]:
p.__dict__['name'] = 'John'

In [34]:
p.__dict__

{'_name': 'Alex', 'name': 'John'}

As you can see, we now have `name` in our instance dictionary.

Let's retrieve the `name` via dotted notation:

In [35]:
p.name

'Alex'

That's obviously still using the getter method.

And setting the name:

In [36]:
p.name = 'Raymond' # p.set_name('Ray')

In [37]:
p.__dict__

{'_name': 'Raymond', 'name': 'John'}

As you can see, it used the setter method.

And the same thing happens with `setattr` and `getattr`:

In [ ]:
getattr(p, 'name')

In [ ]:
setattr(p, 'name', 'Python')

In [ ]:
p.__dict__

In [ ]:
p.name

As you can see the `setattr` method used the property setter.

For completeness, let's see how the deleter method works:

In [44]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        print('getting name')
        return self._name
    
    def set_name(self, value):
        print('setting name')
        self._name = value
        
    def del_name(self):
        print('deleting name')
        del self._name  # or whatever "cleanup" we want to do
        
    name = property(get_name, set_name, del_name)

In [45]:
p = Person('Alex')

In [46]:
p.__dict__

{'_name': 'Alex'}

In [47]:
p.name

getting name


'Alex'

In [48]:
p.name = 'Eric'

setting name


In [49]:
p.__dict__

{'_name': 'Eric'}

In [50]:
del p.name

deleting name


In [51]:
p.__dict__

{}

In [52]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              'get_name': <function __main__.Person.get_name(self)>,
              'set_name': <function __main__.Person.set_name(self, value)>,
              'del_name': <function __main__.Person.del_name(self)>,
              'name': <property at 0x1805a435800>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Now, the property still exists (since it is defined in the class) - all we did was remove the underlying value for the property (the `_name` instance attribute):

In [ ]:
try:
    p.name
except AttributeError as ex:
    print(ex)

As you can see the issue is that the getter function is trying to find `_name` in the attribute, which no longer exists. So the getter and setter still exist (i.e. the property still exists), so we can still assign to it:

In [ ]:
p.name = 'Alex'

In [ ]:
p.name

The last param in `property` is just a docstring. So we could do this:

In [56]:
class Person:
    """This is a Person object"""
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        '''docstring'''
        return self._name
    
    def set_name(self, value):
        self._name = value
        
    name = property(get_name, set_name, doc="The person's name.")

In [60]:
import networkx as nx
nx.Graph?


In [54]:
p = Person('Alex')

In [55]:
help(Person.name)

Help on property:

    The person's name.



In [ ]:
help(Person)

In [61]:
a = 0.1
b = 0.2
print(a+b == 0.3)

False


In [62]:
s = 'print(2)'

In [63]:
eval(s)

2


In [64]:
exec(s)

2


In [69]:
s = 'a = lambda x: x+2'

In [70]:
exec(s)

In [71]:
a(2)

4